`uarray` is a generic override framework for objects and methods in Python. Since [my last `uarray` blogpost](https://labs.quansight.org/blog/2019/04/uarray-intro/), there have been plenty of developments, changes to the API and improvements to the overhead of the protocol. Let’s begin with a walk-through of the current feature set and API, and then move on to current developments and how it compares to [`__array_function__`](www.numpy.org/neps/nep-0018-array-function-protocol.html). For further details on the API and latest developments, please see [the API page for `uarray`](https://uarray.readthedocs.io/en/latest/generated/uarray.html). The examples there are doctested, so they will always be current.

In [1]:
!export NUMPY_EXPERIMENTAL_ARRAY_FUNCTION=1

In [2]:
import uarray as ua
import numpy as np

In `uarray`, the fundamental building block is a multimethod. Multimethods have a number of nice properties, such as automatic dispatch based on backends. It is important to note here that multimethods will be written by API authors, rather than implementors. Here's how we define a multimethod in `uarray`:
<!-- TEASER_END -->

In [3]:
my_multimethod = ua.generate_multimethod(
    # This is the argument extractor, it also defines the signature.
    lambda a, b=None: (),
    # This is the reverse dispatcher, it is important for conversion/coercion. It is optional.
    lambda a, kw, d: (a, kw),
    # This is the domain, it separates the multimethods into clean parts.
    "ql_blogpost",
    # This is the default implementation. It is also optional, which means "error if no backend is found/set".
    default=lambda a, b=None: (a, b)  # This is the optional default
)

We will explore the function of each of the parts of this multimethod later. Let's try calling this multimethod.

In [4]:
my_multimethod(1, 2)

(1, 2)

As we can see, it remains faithful to its purpose. It has a default implementation that it can produce nicely. However, the real power comes when overriding this multimethod. To do this, we must consider the concept of a backend, which is separate. This is different from the view of `__array_function__`, in which array objects themselves define the backend. Here, we have applied the principle of [separation of concerns](https://en.wikipedia.org/wiki/Separation_of_concerns) to separate the multimethod, the objects it operates on, as well as the backend. Note as well that backend authors provide the implementation of a given API.

Let’s see how one would define a backend in `uarray`.

In [5]:
class Backend:
    # A backend can only override methods from its own domain
    __ua_domain__ = "ql_blogpost"
    
    # This is the main protocol a backend must have in order to work.
    @staticmethod
    def __ua_function__(
        func,  # It gets the multimethod being called,
        args, kwargs  # And the arguments the method is called with.
    ):
        # Here we have the implementation
        return func.__name__, args, kwargs


Now, let's go about overriding the function. Note here that `set_backend` will typically be used by consumers of the API.

In [6]:
with ua.set_backend(Backend):
    print(my_multimethod(1, 2))

('<lambda>', (1, 2), {})


As you can see, the function's return value magically changed. Note that this propagates all the way down the call stack. With that, let's get into some of the more magical features of `uarray`, starting with the function of the argument extractor, the argument replacer, and the `__ua_convert__` protocol.

## The argument extractor and argument-based dispatch

The argument extractor (equivalent to a dispatcher in `__array_function__`) is more than just a dummy that returns an empty tuple. It can also return the arguments needed for dispatch. Let’s go ahead and see this in action.

In [7]:
import numbers
my_multimethod_with_dispatch = ua.generate_multimethod(
    # a is dispatchable, and it's supposed to be some kind of number
    lambda a, b=None: (ua.Dispatchable(a, numbers.Number),),
    lambda a, kw, d: (a, kw),
    "ql_blogpost"
)

Just to illustrate what happens when a multimethod doesn't have a default implementation, let’s call this multimethod.

In [8]:
my_multimethod_with_dispatch(1, 2)

BackendNotImplementedError: ('No selected backends had an implementation for this function.', [])

It raises an error, as is expected, with a message explaining the situation. Now, let's explore dispatch.

In [9]:
class DispatchBackendInt:
    # A backend can only override methods from its own domain
    __ua_domain__ = "ql_blogpost"
    
    @staticmethod
    def __ua_function__(func, args, kwargs):
        return "int", func.__name__, args, kwargs
    
    @staticmethod
    def __ua_convert__(
        dispatchables,  # The list of dispatchables
        coerce # Whether or not to forcibly coerce them to the required form, if possible
    ):
        converted = []
        for d in dispatchables:
            # Check if it's a number, we only support ints
            if d.type is numbers.Number and isinstance(d.value, int):
                converted.append(d.value)
            else:
                return NotImplemented
        
        return converted

class DispatchBackendFloat:
    # A backend can only override methods from its own domain
    __ua_domain__ = "ql_blogpost"
    
    @staticmethod
    def __ua_function__(func, args, kwargs):
        return "float", func.__name__, args, kwargs
    
    @staticmethod
    def __ua_convert__(dispatchables, coerce):
        converted = []
        for d in dispatchables:
            # This one only supports floats
            if d.type is numbers.Number and isinstance(d.value, float):
                converted.append(d.value)
            else:
                return NotImplemented
        
        return converted

In [10]:
with ua.set_backend(DispatchBackendInt), ua.set_backend(DispatchBackendFloat):
    print(my_multimethod_with_dispatch(1, 2))
    print(my_multimethod_with_dispatch(1.0, 2))
    print(my_multimethod_with_dispatch("1", 2))

('int', '<lambda>', (1, 2), {})
('float', '<lambda>', (1.0, 2), {})


BackendNotImplementedError: ('No selected backends had an implementation for this function.', [])

As we can see, the method dispatches fine on `int` and `float` to the correct implementation but raises an error for strings. Let's make a small modification to make even working with `int` possible:

In [11]:
# The argument replacer is supposed to replace the args/kwargs
# with the right dispatchables
def my_ar(args, kwargs, dispatchables):
    def replacer(a, b=None):
        return dispatchables, {'b': b}
    
    return replacer(*args, **kwargs)

my_multimethod_with_dispatch = ua.generate_multimethod(
    lambda a, b=None: (ua.Dispatchable(a, numbers.Number),),
    my_ar,  # We put the right thing here.
    "ql_blogpost"
)

In [12]:
class DispatchBackendInt:
    # A backend can only override methods from its own domain
    __ua_domain__ = "ql_blogpost"
    
    @staticmethod
    def __ua_function__(func, args, kwargs):
        return "int", func.__name__, args, kwargs
    
    @staticmethod
    def __ua_convert__(dispatchables, coerce):
        converted = []
        for d in dispatchables:
            if d.type is numbers.Number:
                if isinstance(d.value, int):
                    converted.append(d.value)
                # If we're allowed to coerce it
                elif coerce and d.coercible:
                    try:
                        converted.append(int(d.value))
                    # Make sure unsupported conversions are caught
                    except TypeError:
                        return NotImplemented
                else:
                    return NotImplemented
            else:
                # Handle the base case
                return NotImplemented
        
        return converted

In [13]:
with ua.set_backend(DispatchBackendInt, coerce=True):
    print(my_multimethod_with_dispatch("1", 2))

('int', '<lambda>', (1,), {'b': 2})


Note that we must pass the `coerce=True` parameter to make this work, otherwise the method will return `NotImplemented` and fail. Note also that the string has been correctly converted before being passed into the function. I previously mentioned the possibility of adding a reverse dispatcher to `__array_function__` [[1]](https://github.com/numpy/numpy/issues/12974#issuecomment-467197477) [[2]](https://github.com/dask/dask/pull/4567#issuecomment-471630100), but it was rejected. I have added it to `uarray`, but it is optional.

## Meta-backends: Backends which may rely on others
One other thing easily possible in `uarray` not easily possible in other frameworks is the possibility of meta backends. These are backends which could rely on other backends. Suppose, that a backend wraps other objects and needs to get data out, and then call other backends:

In [14]:
class CustomNumber(numbers.Number):
    def __init__(self, n):
        self.n = n

In [15]:
class DispatchBackendCustom:
    # A backend can only override methods from its own domain
    __ua_domain__ = "ql_blogpost"
    
    @staticmethod
    def __ua_function__(func, args, kwargs):
        with ua.skip_backend(DispatchBackendCustom):
            return ("Custom",) + func(*args, **kwargs)
            
    @staticmethod
    def __ua_convert__(dispatchables, coerce):
        converted = []
        for d in dispatchables:
            if d.type is numbers.Number and isinstance(d.value, CustomNumber):
                # Access the internal value, no conversion going on
                converted.append(d.value.n)
            else:
                # Handle the base case
                return NotImplemented
        
        return converted

In [16]:
with ua.set_backend(DispatchBackendCustom), ua.set_backend(DispatchBackendInt):
    print(my_multimethod_with_dispatch(CustomNumber(1), 2))

('Custom', 'int', '<lambda>', (1,), {'b': 2})


## Permanent registration of backend
`uarray` provides permanent registration for backends, after which the backend will be automatically tried every time. This is user-facing code, and we recommend that no libraries register themselves permanently, apart from reference implementations. This can be done via the [`register_backend`](https://uarray.readthedocs.io/en/latest/generated/uarray.register_backend.html) method.

## The overhead of `uarray`

One of the limiting factors in the adoption of `uarray` has been its overhead. This is currently [being brought down](https://github.com/Quansight-Labs/uarray/pull/170) by a GSoC student working on `uarray`, named [Peter Bell](https://github.com/peterbell10), who is working on a C++ implementation of the protocol. As of the last version, the overhead on my machine was about 1.37 μs per call, and this is still being worked on.